# Skip-gram word2vec

In this notebook, I'll lead you through using TensorFlow to implement the word2vec algorithm using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like machine translation.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* An [implementation of word2vec](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) from Thushan Ganegedara
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)

## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of classes to predict, one for each word. Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other 50,000 set to 0. The matrix multiplication going into the first hidden layer will have almost all of the resulting values be zero. This a huge waste of computation. 

![one-hot encodings](assets/one_hot_encoding.png)

To solve this problem and greatly increase the efficiency of our networks, we use what are called embeddings. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

![lookup](assets/lookup_matrix.png)

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=500>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix as well.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.



## Word2Vec

The word2vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words. Words that show up in similar contexts, such as "black", "white", and "red" will have vectors near each other. There are two architectures for implementing word2vec, CBOW (Continuous Bag-Of-Words) and Skip-gram.

<img src="assets/word2vec_architectures.png" width="500">

In this implementation, we'll be using the skip-gram architecture because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

First up, importing packages.

In [1]:
import time

import numpy as np
import tensorflow as tf

import utils

Load the [text8 dataset](http://mattmahoney.net/dc/textdata.html), a file of cleaned up Wikipedia articles from Matt Mahoney. The next cell will download the data set to the `data` folder. Then you can extract it and delete the archive file to save storage space.

In [2]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'data'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)
        
with open('data/text8') as f:
    text = f.read()

## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [3]:
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [4]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


And here I'm creating dictionaries to convert words to integers and backwards, integers to words. The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1 and so on. The words are converted to integers and stored in the list `int_words`.

In [5]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

I'm going to leave this up to you as an exercise. This is more of a programming challenge, than about deep learning specifically. But, being able to prepare your data for your network is an important skill to have. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [6]:
print(int_words[0:10])

[5239, 3080, 11, 5, 194, 1, 3136, 45, 58, 155]


In [26]:
## Your code here
from collections import Counter
import random
word_counts = Counter()
t = 1e-5#threshold value
tot_words = len(int_words)
# count = collections.Counter(int_words)
for i in int_words:
    word_counts[i] +=1


In [29]:
freq = {}
p_drop = {}
train_words = []
for word,count in word_counts.items():
    freq[word] = count/tot_words
    
for word in word_counts:
    p_drop[word] = 1-np.sqrt(t/freq[word])
    
for word in int_words:
    if random.random()<(1-p_drop[word]):
        train_words.append(word)
        
# I like to think what is commented out below will work...
# train_words = []
# for key,value in count.items():
#     freq_temp = count[key]/tot_words
#     prob_temp = 1-np.sqrt(t/freq_temp)
#     if prob_temp > random.random():
#         train_words.append(key)# The final subsampled word list

## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $< 1; C >$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you choose a random number of words from the window.

In [31]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    # Your code here
#     R = np.random.randint(1,window_size+1)# the random number between 1 and window_size
#     if idx-R > 0:
#         start = idx-R
#     else:
#         start = 0
#     stop = idx+R
#     word_out = set(words[start:idx]+words[idx+1:stop+1])#put in set to get rid of duplicates
#     return list(word_out)#have to convert from set to list for return

    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = set(words[start:idx] + words[idx+1:stop+1])
    return list(target_words)

Here's a function that returns batches for our network. The idea is that it grabs `batch_size` words from a words list. Then for each of those words, it gets the target words in the window. I haven't found a way to pass in a random number of target words and get it to work with the architecture, so I make one row per input-target pair. This is a generator function by the way, helps save memory.

In [32]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

## Building the graph

From [Chris McCormick's blog](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/), we can see the general structure of our network.
![embedding_network](./assets/skip_gram_net_arch.png)

The input words are passed in as integers. This will go into a hidden layer of linear units, then into a softmax layer. We'll use the softmax layer to make a prediction like normal.

The idea here is to train the hidden layer weight matrix to find efficient representations for our words. We can discard the softmax layer becuase we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in other networks we build from the dataset.

I'm going to have you build the graph in stages now. First off, creating the `inputs` and `labels` placeholders like normal.

> **Exercise:** Assign `inputs` and `labels` using `tf.placeholder`. We're going to be passing in integers, so set the data types to `tf.int32`. The batches we're passing in will have varying sizes, so set the batch sizes to [`None`]. To make things work later, you'll need to set the second dimension of `labels` to `None` or `1`.

In [33]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32,shape=[None],name='inputs')
    labels = tf.placeholder(tf.int32, shape = [None,None],name='labels')

## Embedding



The embedding matrix has a size of the number of words by the number of units in the hidden layer. So, if you have 10,000 words and 300 hidden units, the matrix will have size $10,000 \times 300$. Remember that we're using tokenized data for our inputs, usually as integers, where the number of tokens is the number of words in our vocabulary.


> **Exercise:** Tensorflow provides a convenient function [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) that does this lookup for us. You pass in the embedding matrix and a tensor of integers, then it returns rows in the matrix corresponding to those integers. Below, set the number of embedding features you'll use (200 is a good start), create the embedding matrix variable, and use `tf.nn.embedding_lookup` to get the embedding tensors. For the embedding matrix, I suggest you initialize it with a uniform random numbers between -1 and 1 using [tf.random_uniform](https://www.tensorflow.org/api_docs/python/tf/random_uniform).

In [34]:
n_vocab = len(int_to_vocab)
n_embedding = 200 # Number of embedding features 
with train_graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_vocab,n_embedding),minval = -1,maxval=-1))####### create embedding weight matrix here
    embed = tf.nn.embedding_lookup(embedding,inputs)# use tf.nn.embedding_lookup to get the hidden layer output

## Negative sampling



For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct label, but only a small number of incorrect labels. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). Tensorflow has a convenient function to do this, [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss).

> **Exercise:** Below, create weights and biases for the softmax layer. Then, use [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss) to calculate the loss. Be sure to read the documentation to figure out how it works.

In [35]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w =tf.Variable(tf.truncated_normal(shape = (n_vocab,n_embedding),mean=0,stddev=0.1)) # create softmax weight matrix here
    softmax_b = tf.Variable(tf.zeros(n_vocab)) # create softmax biases here
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w,softmax_b,labels=labels,inputs=embed,num_sampled=n_sampled,num_classes=n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

## Validation

This code is from Thushan Ganegedara's implementation. Here we're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [36]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [37]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints

mkdir: checkpoints: File exists


## Training

Below is the code to train the network. Every 100 batches it reports the training loss. Every 1000 batches, it'll print out the validation words.

In [38]:
epochs = 10
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                ## From Thushan Ganegedara's implementation
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/10 Iteration: 100 Avg. Training loss: 5.6488 0.3603 sec/batch
Epoch 1/10 Iteration: 200 Avg. Training loss: 6.3790 0.3917 sec/batch
Epoch 1/10 Iteration: 300 Avg. Training loss: 7.6658 0.4109 sec/batch
Epoch 1/10 Iteration: 400 Avg. Training loss: 8.9547 0.3443 sec/batch
Epoch 1/10 Iteration: 500 Avg. Training loss: 10.0614 0.3736 sec/batch
Epoch 1/10 Iteration: 600 Avg. Training loss: 10.6870 0.3233 sec/batch
Epoch 1/10 Iteration: 700 Avg. Training loss: 11.2076 0.3583 sec/batch
Epoch 1/10 Iteration: 800 Avg. Training loss: 11.9779 0.4893 sec/batch
Epoch 1/10 Iteration: 900 Avg. Training loss: 12.1117 0.3623 sec/batch
Epoch 1/10 Iteration: 1000 Avg. Training loss: 12.8477 0.4814 sec/batch
Nearest to but: byword, ibook, repels, defraud, alexandrovich, pilkington, undersecretary, sanusi,
Nearest to about: avesta, chloe, gunboat, workingmen, artbook, hyperion, iers, corundum,
Nearest to these: syne, ghouls, hennecke, apokryphen, pathfinder, apok, studia, hochdeutsch,
Nearest to t

Epoch 1/10 Iteration: 4100 Avg. Training loss: 13.1739 0.2934 sec/batch
Epoch 1/10 Iteration: 4200 Avg. Training loss: 12.2225 0.2933 sec/batch
Epoch 1/10 Iteration: 4300 Avg. Training loss: 12.1820 0.2961 sec/batch
Epoch 1/10 Iteration: 4400 Avg. Training loss: 12.0561 0.2931 sec/batch
Epoch 1/10 Iteration: 4500 Avg. Training loss: 12.4014 0.2879 sec/batch
Epoch 1/10 Iteration: 4600 Avg. Training loss: 12.3486 0.2901 sec/batch
Epoch 2/10 Iteration: 4700 Avg. Training loss: 11.9334 0.2120 sec/batch
Epoch 2/10 Iteration: 4800 Avg. Training loss: 11.9055 0.2989 sec/batch
Epoch 2/10 Iteration: 4900 Avg. Training loss: 12.4025 0.2945 sec/batch
Epoch 2/10 Iteration: 5000 Avg. Training loss: 11.6185 0.2948 sec/batch
Nearest to but: by, is, this, a, that, for, have, not,
Nearest to about: such, was, and, elsewhere, founder, route, as, estimated,
Nearest to these: however, as, not, without, including, hundreds, or, there,
Nearest to that: a, of, was, for, it, the, not, is,
Nearest to so: profe

Epoch 2/10 Iteration: 8800 Avg. Training loss: 8.3514 0.2913 sec/batch
Epoch 2/10 Iteration: 8900 Avg. Training loss: 8.1084 0.2917 sec/batch
Epoch 2/10 Iteration: 9000 Avg. Training loss: 8.1635 0.2926 sec/batch
Nearest to but: to, have, it, that, a, this, the, of,
Nearest to about: and, in, from, by, also, the, s, of,
Nearest to these: as, there, or, that, but, are, this, the,
Nearest to that: the, not, a, to, as, it, of, is,
Nearest to so: that, they, or, not, a, be, the, which,
Nearest to also: a, to, of, the, and, in, as, for,
Nearest to known: the, and, of, in, but, a, by, as,
Nearest to of: the, a, in, and, by, for, to, from,
Nearest to report: changed, s, after, single, reported, again, caused, granted,
Nearest to existence: outside, being, brief, considerable, why, finally, independent, others,
Nearest to professional: worldwide, bring, getting, devoted, studies, taking, similarly, except,
Nearest to account: according, on, beginning, was, found, life, position, more,
Nearest 

Epoch 3/10 Iteration: 13100 Avg. Training loss: 7.2505 0.2931 sec/batch
Epoch 3/10 Iteration: 13200 Avg. Training loss: 6.3582 0.2909 sec/batch
Epoch 3/10 Iteration: 13300 Avg. Training loss: 7.1114 0.2886 sec/batch
Epoch 3/10 Iteration: 13400 Avg. Training loss: 6.5867 0.2881 sec/batch
Epoch 3/10 Iteration: 13500 Avg. Training loss: 6.3970 0.2932 sec/batch
Epoch 3/10 Iteration: 13600 Avg. Training loss: 6.3851 0.2898 sec/batch
Epoch 3/10 Iteration: 13700 Avg. Training loss: 6.3198 0.2904 sec/batch
Epoch 3/10 Iteration: 13800 Avg. Training loss: 6.4624 0.2901 sec/batch
Epoch 4/10 Iteration: 13900 Avg. Training loss: 6.5601 0.0469 sec/batch
Epoch 4/10 Iteration: 14000 Avg. Training loss: 6.4645 0.2906 sec/batch
Nearest to but: that, have, to, a, because, this, as, just,
Nearest to about: from, also, and, since, the, of, in, by,
Nearest to these: because, possible, different, that, such, do, this, or,
Nearest to that: not, it, be, this, would, but, have, to,
Nearest to so: not, that, or,

Epoch 4/10 Iteration: 17600 Avg. Training loss: 5.8906 0.2940 sec/batch
Epoch 4/10 Iteration: 17700 Avg. Training loss: 6.3306 0.2900 sec/batch
Epoch 4/10 Iteration: 17800 Avg. Training loss: 5.7212 0.2916 sec/batch
Epoch 4/10 Iteration: 17900 Avg. Training loss: 6.0711 0.2910 sec/batch
Epoch 4/10 Iteration: 18000 Avg. Training loss: 6.1884 0.2901 sec/batch
Nearest to but: that, have, while, to, their, only, this, as,
Nearest to about: and, from, also, for, the, in, by, with,
Nearest to these: such, or, different, are, because, possible, effect, generally,
Nearest to that: not, it, be, this, person, should, so, rather,
Nearest to so: that, be, not, do, or, explain, it, simply,
Nearest to also: a, for, developed, as, about, and, most, with,
Nearest to known: in, and, as, the, from, of, well, name,
Nearest to of: the, in, and, by, from, as, known, to,
Nearest to report: on, commission, reports, department, announced, nationwide, world, news,
Nearest to existence: idea, ought, belief, not

Epoch 5/10 Iteration: 21900 Avg. Training loss: 5.3961 0.3274 sec/batch
Epoch 5/10 Iteration: 22000 Avg. Training loss: 5.1374 0.3099 sec/batch
Nearest to but: it, that, as, not, only, to, so, have,
Nearest to about: and, also, roughly, from, people, of, demography, accounts,
Nearest to these: such, reconstruct, or, some, different, generally, certain, whereas,
Nearest to that: not, it, selfish, ought, argued, should, truly, rather,
Nearest to so: be, that, it, not, this, but, either, or,
Nearest to also: as, most, other, includes, see, well, about, of,
Nearest to known: in, as, and, from, found, parts, earliest, name,
Nearest to of: the, and, in, by, from, established, constituted, kingship,
Nearest to report: reports, coverage, commission, department, international, agency, surveillance, registration,
Nearest to existence: belief, notion, conceptions, ought, truths, assert, argument, idea,
Nearest to professional: sports, sport, teams, football, sporting, leagues, soccer, athletics,


Epoch 6/10 Iteration: 25700 Avg. Training loss: 4.9978 0.3142 sec/batch
Epoch 6/10 Iteration: 25800 Avg. Training loss: 4.7878 0.3652 sec/batch
Epoch 6/10 Iteration: 25900 Avg. Training loss: 5.0487 0.3255 sec/batch
Epoch 6/10 Iteration: 26000 Avg. Training loss: 5.1783 0.3623 sec/batch
Nearest to but: it, that, not, because, to, so, only, they,
Nearest to about: of, very, roughly, estimating, discrepancy, what, people, why,
Nearest to these: different, such, other, reconstruct, salient, as, immutable, frameworks,
Nearest to that: not, it, ought, should, be, but, selfish, any,
Nearest to so: be, that, not, it, but, they, say, to,
Nearest to also: includes, other, see, several, most, article, as, and,
Nearest to known: of, refer, in, loosely, as, earliest, etymologies, modern,
Nearest to of: the, in, and, from, by, chronologically, known, genealogy,
Nearest to report: news, reports, review, reporting, assessment, coverage, surveillance, gov,
Nearest to existence: notion, conceptions, on

Epoch 7/10 Iteration: 29200 Avg. Training loss: 4.9225 0.3049 sec/batch
Epoch 7/10 Iteration: 29300 Avg. Training loss: 4.9255 0.3224 sec/batch
Epoch 7/10 Iteration: 29400 Avg. Training loss: 4.7716 0.3142 sec/batch
Epoch 7/10 Iteration: 29500 Avg. Training loss: 4.8022 0.3104 sec/batch
Epoch 7/10 Iteration: 29600 Avg. Training loss: 4.8287 0.3204 sec/batch
Epoch 7/10 Iteration: 29700 Avg. Training loss: 5.0904 0.3309 sec/batch
Epoch 7/10 Iteration: 29800 Avg. Training loss: 4.6442 0.3080 sec/batch
Epoch 7/10 Iteration: 29900 Avg. Training loss: 4.6936 0.3206 sec/batch
Epoch 7/10 Iteration: 30000 Avg. Training loss: 4.8141 0.3300 sec/batch
Nearest to but: it, that, not, to, because, so, however, though,
Nearest to about: why, what, of, really, how, view, and, estimating,
Nearest to these: such, most, as, different, other, some, delineated, reconstruct,
Nearest to that: not, it, this, therefore, be, so, to, possible,
Nearest to so: be, that, not, it, we, to, will, can,
Nearest to also: 

KeyboardInterrupt: 

Restore the trained network if you need to:

In [39]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    embed_mat = sess.run(embedding)

ValueError: Can't load save_path when it is None.

## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)